Name

Labpartner(s)

In [ ]:
#import statements go here

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Class 10.2

# Warmups 10.2

If you do not complete the warmup during the class period, complete for homework and review/expand for weekly practice. These are basic coding skills you need to be comfortable with. Work on the following for 25 min and then we will go over them together:

**W.1** Write a function that returns a list of the first n powers of 2 (2^0, 2^1, 2^2, 2^3...2^n).

**W.2** Using the function above, plot the first 100 powers of 2

# Lecture 10.1

### Agenda:
- Announcements
- Correlation 
- Introduction to fittng models to data

#### Announcements
- Congratulations to everyone who got over 20,000 points on data camp! +5 tokens for all of you. For those of you catching up, let me know if you reach this. There will be pizza and other prizes for the top token folks.
- August figured out the issue with making all the frames the same length in Davinci Resolve +2 tokens
- Reports that the last few datacamp chapters were unclear. What are good strategies for dealing with this?


### Correlation

### A note of caution:

"Correlation does not imply causation"

Example:
As ice cream sales increase, the rate of drowning deaths increases sharply.
Therefore, ice cream consumption causes drowning.

When we get data, we want to describe how variables relate to each other. The simplest question we can ask are two variables correlated, that is do they change togehter?

The metric we use for correlation is the Pearson correlation coefficient, which is hard to grasp intuitively from looking at the equation (see https://en.wikipedia.org/wiki/Correlation_and_dependence). Basically, correlation is a measure of how linearly correlated two variables are, and has values between 1 and -1. If the two variables fall on a straight line and the both increase together, then the correlation is 1. If one increases while the other decreases, the correlation is -1. Correlations near zero mean the two variables are not correlated. 

Let's use some movie data to test if budgets and gross earnings are correlated. This is from the pandas tutorial here https://www.dataquest.io/blog/excel-and-pandas/ which uses and IMDB dataset. See also the lectures from this class in Fall 2019 

You can download the data file on the above website.

In [ ]:
file = "movies.xls"
movies_sheet1 = pd.read_excel(file, sheet_name=0, index_col=0)
movies_sheet2 = pd.read_excel(file, sheet_name=1, index_col=0)
movies_sheet3 = pd.read_excel(file, sheet_name=2, index_col=0)
movies = pd.concat([movies_sheet1, movies_sheet2, movies_sheet3])

In [ ]:
movies # over 5000 movies

In [ ]:
# let's plot earnings vs. budget and a 1 to 1 line

# plot Budget vs Gross Earnings
plt.scatter(movies['Budget'], movies['Gross Earnings'])
plt.plot([0,1E10],[0,1E10], 'k--')
plt.xlabel('Budget')
plt.ylabel('Gross Earnings')
plt.xlim([0,0.1E10])
plt.ylim([0,1E9])

In the above I have added the y=x line to visualize if the gross earnings were above the budget. 
Because many of the data points are on top of each other, it is difficult to assess visually the relationship between the two variables

Note that pandas allows us to get the correlation of the entire dataframe all at once. There are many other packages that will do this as well, including numpy.

In [ ]:
movies.corr()
# Budget and Gross earnings are not very correlated, r = 0.10

### Introduction to linear regression

See https://en.wikipedia.org/wiki/Linear_regression and http://onlinestatbook.com/2/regression/regression.html for a nice introduction. See https://towardsdatascience.com/linear-regression-using-python-b136c91bf0a2 for a good intro on the math

Supposed we were convinced that gross earnings were linearly related to budget, i.e. G = a*B +c. To test this, we would fit the scatterplot we made above to a line, which would solve for the coefficients a (slope) and c (intercept) in our linear equation.

The basic idea of linear regression is that one finds the distance of each data point to a line and minimize this (using calculus) to find the best fit line. This is commonly called ordinary least squares, as the distance of the data from the linear model is squared.

Because the movies database has a number of NaNs (not a number, or missing values) in it, it's a bit difficult to work with (but see tips on revmoving NaNs in the resources page in github). Instead, let's do the tutorial here on linear regression using Boston housing data: https://towardsdatascience.com/linear-regression-on-boston-housing-dataset-f409b7e4a155 This tutorial uses the package scikit-learn, which is a collection of advanced data analysis and machine learning functions. We will use this again in the future. Scikit-learn comes with a number of datasets to practice with, the housing data is one of them.


In [ ]:
from sklearn import datasets ## imports datasets from scikit-learn
data = datasets.load_boston() ## loads Boston dataset from datasets library 

What we want to do here is to determine which variables contribute to housing prices

In [ ]:
data?
# this is a new type of thing, a bunch

In [ ]:
# information on the data:
print(data.DESCR)

It is really common in scikitlearn to break the data up into a target dataset, the thing one is trying to predict, and all the factors that may affect that target variable

In [ ]:
# convert the data set into a pandas data frame  
df = pd.DataFrame(data.data, columns=data.feature_names)

# Put the target (housing value -- MEDV) in another DataFrame
target = pd.DataFrame(data.target, columns=["MEDV"])

Create a second dataframe with the housing value added as a column. 
What variables correlate most with housing value? Hypothesize why this might be.

In [ ]:
# create a second dataframe that has MEDV in it so we can look at correlation`
df2 = df
df2['MEDV'] = target.MEDV

In [ ]:
df2.corr()

LSTAT, % lower status of the population, correlates negatively, which is not surprising.

RM, the number of rooms, has the highest positive correlation

In [ ]:
# are room numbers and value correlated?
np.corrcoef(df["RM"], target["MEDV"])

# note this is a matrix of correlation coefficients, you only need the top right one

In [ ]:
plt.scatter(df["RM"], target["MEDV"])
plt.xlabel('Number of Rooms')
plt.ylabel('House value ($1000s)')

### Doing a linear regression using scipy

In [ ]:
from scipy import stats

In [ ]:
stats.linregress?

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(df["RM"],target["MEDV"])

this gives us the slope and intercept of the line, the corrlation (r_value)

A low p-value (< 0.05) indicates that the linear regression is a good model

the standard error is the standard deviation of the data from the model (as opposed to the standard deviation from the mean of the data)

In [ ]:
slope

In [ ]:
intercept

In [ ]:
r_value
# note this is the same correlation value we got before

In [ ]:
p_value
# is this good? Do we have confidence in our linear model?

In [ ]:
std_err

In [ ]:
# let's plot the data with the linear fit and see how it looks
plt.plot(df["RM"], slope*df["RM"]+intercept, 'k--')
plt.scatter(df["RM"], target["MEDV"])
plt.xlabel('Number of Rooms')
plt.ylabel('House value ($1000s)')

### Doing a linear regression using statsmodels

Statsmodels is a powerful statistics package that is very popular

See https://towardsdatascience.com/simple-and-multiple-linear-regression-in-python-c928425168f9

and

https://www.statsmodels.org/dev/index.html

http://www.statsmodels.org/dev/examples/index.html#linear-regression-models

In [ ]:
import statsmodels.api as sm

In [ ]:
# look at the helpfiles for OLS, ordinary least squares fitting, aka linear fitting
help(sm.OLS)

In [ ]:
# same data as before:
X = df["RM"] # what we think the cost depends on, rooms
y = target["MEDV"]

In [ ]:
# Note y, the target of what we are fitting to, goes first:
model = sm.OLS(y, X).fit()

# this is fitting y = aX, with no constant, i.e. using an intercept of zero

In [ ]:
model.params

In [ ]:
model.summary()

In [ ]:
predictions = model.predict(X) # make the predictions by the model

In [ ]:
# let's plot the result like we did before

plt.plot(X,predictions, 'k--')
plt.scatter(df["RM"], target["MEDV"])
plt.xlabel('Number of Rooms')
plt.ylabel('House value ($1000s)')

# this is not a very good fit, because we assumed the intercept of the line was zero

# Lab 10.2

**E.1** What kind of thing is movies? How do you know?

**E.2**  What does it mean that year and duration in movies are negatively correlated? Plot these two variables togther and explain.

**E.3** Which pairs of variables in the movies dataset have the highest correlation? What might explain this?

**E.4** Remake the house value vs. number of rooms plot and write the equation of the fitted line the title or as an annotation

**E.5** Following the same method, make a linear model for the house value as a function of the LSTAT (% lower status of the population). Plot your result and the resulting model, and annotate your result with the model equation. How good is this model compared to the one with the number of rooms? Use the statistcs returned by the linear regression function to support your argument.

**E.6** Following the example in the documentation below, add an intercept to the statsmodels housing model and plot your result

http://www.statsmodels.org/dev/examples/notebooks/generated/ols.html

**E.7** (Advanced/optional, +1 token) Do the tutorial here to use scikitlearn to do a linear model: https://towardsdatascience.com/linear-regression-on-boston-housing-dataset-f409b7e4a155